In [1]:
import pandas as pd

In [2]:
file_path = "HousingStarted_Raw.csv"
df = pd.read_csv(file_path)

In [3]:
df.head()

,REF_DATE,GEO,DGUID,Housing estimates,Type of unit,Seasonal adjustment,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1948-01,Canada,2016A000011124,Housing starts,Total units,Unadjusted,Units,300,units,0,v730416,1.1.1.1,6678.0,NaN,NaN,NaN,0
1,1948-01,Canada,2016A000011124,Housing under construction,Total units,Unadjusted,Units,300,units,0,v731381,1.2.1.1,37529.0,NaN,NaN,NaN,0
2,1948-01,Canada,2016A000011124,Housing completions,Total units,Unadjusted,Units,300,units,0,v732318,1.3.1.1,11569.0,NaN,NaN,NaN,0
3,1948-01,Prince Edward Island,2016A000211,Housing starts,Total units,Unadjusted,Units,300,units,0,v730419,4.1.1.1,0.0,NaN,NaN,NaN,0
4,1948-01,Prince Edward Island,2016A000211,Housing under construction,Total units,Unadjusted,Units,300,units,0,v731384,4.2.1.1,173.0,NaN,NaN,NaN,0


In [4]:
# List of columns to remove
columns_to_remove = [
    "DGUID", "Seasonal adjustment", "UOM", "UOM_ID", 
    "SCALAR_FACTOR", "SCALAR_ID", "VECTOR", "COORDINATE", "STATUS", 
    "SYMBOL", "TERMINATED", "DECIMALS"
]

# Drop the specified columns
df.drop(columns=columns_to_remove, inplace=True, errors='ignore')

In [5]:
df.head()

,REF_DATE,GEO,Housing estimates,Type of unit,VALUE
0,1948-01,Canada,Housing starts,Total units,6678.0
1,1948-01,Canada,Housing under construction,Total units,37529.0
2,1948-01,Canada,Housing completions,Total units,11569.0
3,1948-01,Prince Edward Island,Housing starts,Total units,0.0
4,1948-01,Prince Edward Island,Housing under construction,Total units,173.0


In [6]:
#Filtering only total units data
df_filtered = df[df["Type of unit"] == "Total units"].drop(columns=["Type of unit"])

In [7]:
df_filtered.head()

,REF_DATE,GEO,Housing estimates,VALUE
0,1948-01,Canada,Housing starts,6678.0
1,1948-01,Canada,Housing under construction,37529.0
2,1948-01,Canada,Housing completions,11569.0
3,1948-01,Prince Edward Island,Housing starts,0.0
4,1948-01,Prince Edward Island,Housing under construction,173.0


In [8]:
# Adding index column
df_filtered.insert(0, 'Index', range(1, len(df_filtered) + 1))

In [9]:
df_filtered.head()

,Index,REF_DATE,GEO,Housing estimates,VALUE
0,1,1948-01,Canada,Housing starts,6678.0
1,2,1948-01,Canada,Housing under construction,37529.0
2,3,1948-01,Canada,Housing completions,11569.0
3,4,1948-01,Prince Edward Island,Housing starts,0.0
4,5,1948-01,Prince Edward Island,Housing under construction,173.0


In [10]:
#pivoting the Housing estimates column
df_pivot = df_filtered.pivot(index=['REF_DATE', 'GEO', 'Index'], columns="Housing estimates", values='VALUE').reset_index()

In [11]:
df_pivot

Housing estimates,REF_DATE,GEO,Index,Housing completions,Housing starts,Housing under construction
0,1948-01,Alberta,25,NaN,250.0,NaN
1,1948-01,Alberta,26,NaN,NaN,1855.0
2,1948-01,Alberta,27,855.0,NaN,NaN
3,1948-01,British Columbia,28,NaN,2121.0,NaN
4,1948-01,British Columbia,29,NaN,NaN,6628.0
...,...,...,...,...,...,...
12255,2024-10,Nova Scotia,12253,NaN,1437.0,NaN
12256,2024-10,Ontario,12256,NaN,16287.0,NaN
12257,2024-10,Prince Edward Island,12252,NaN,357.0,NaN
12258,2024-10,Quebec,12255,NaN,15814.0,NaN


In [12]:
df_pivot.reset_index(drop=True, inplace=True)
df_pivot.columns

Index(['REF_DATE', 'GEO', 'Index', 'Housing completions', 'Housing starts',
       'Housing under construction'],
      dtype='object', name='Housing estimates')

In [13]:
## Group the data by REF_DATE and GEO without setting them as index
grouped_df = df_pivot.groupby(['REF_DATE', 'GEO'], as_index=False).agg({
    'Index': 'first',  
    'Housing completions': 'first',  
    'Housing starts': 'first',  
    'Housing under construction': 'first'  
})

In [14]:
grouped_df

Housing estimates,REF_DATE,GEO,Index,Housing completions,Housing starts,Housing under construction
0,1948-01,Alberta,25,855.0,250.0,1855.0
1,1948-01,British Columbia,28,2189.0,2121.0,6628.0
2,1948-01,Canada,1,11569.0,6678.0,37529.0
3,1948-01,Manitoba,19,655.0,125.0,1785.0
4,1948-01,New Brunswick,10,209.0,14.0,531.0
...,...,...,...,...,...,...
3731,2024-10,Nova Scotia,12253,NaN,1437.0,NaN
3732,2024-10,Ontario,12256,NaN,16287.0,NaN
3733,2024-10,Prince Edward Island,12252,NaN,357.0,NaN
3734,2024-10,Quebec,12255,NaN,15814.0,NaN


In [15]:
grouped_df["REF_DATE"] = pd.to_datetime(grouped_df["REF_DATE"])

In [16]:
grouped_df

Housing estimates,REF_DATE,GEO,Index,Housing completions,Housing starts,Housing under construction
0,1948-01-01,Alberta,25,855.0,250.0,1855.0
1,1948-01-01,British Columbia,28,2189.0,2121.0,6628.0
2,1948-01-01,Canada,1,11569.0,6678.0,37529.0
3,1948-01-01,Manitoba,19,655.0,125.0,1785.0
4,1948-01-01,New Brunswick,10,209.0,14.0,531.0
...,...,...,...,...,...,...
3731,2024-10-01,Nova Scotia,12253,NaN,1437.0,NaN
3732,2024-10-01,Ontario,12256,NaN,16287.0,NaN
3733,2024-10-01,Prince Edward Island,12252,NaN,357.0,NaN
3734,2024-10-01,Quebec,12255,NaN,15814.0,NaN


In [17]:
# Generate a complete monthly date range for each GEO
full_dates = pd.date_range(start=grouped_df['REF_DATE'].min(), end=grouped_df['REF_DATE'].max(), freq='MS')
all_geos = grouped_df['GEO'].unique()

In [18]:
# Create a new DataFrame with all month-wise GEO combinations
full_df = pd.MultiIndex.from_product([full_dates, all_geos], names=['REF_DATE', 'GEO']).to_frame(index=False)

In [19]:
# Merge with original data
grouped_df = full_df.merge(grouped_df, on=['REF_DATE', 'GEO'], how='left')

In [20]:
grouped_df

,REF_DATE,GEO,Index,Housing completions,Housing starts,Housing under construction
0,1948-01-01,Alberta,25.0,855.0,250.0,1855.0
1,1948-01-01,British Columbia,28.0,2189.0,2121.0,6628.0
2,1948-01-01,Canada,1.0,11569.0,6678.0,37529.0
3,1948-01-01,Manitoba,19.0,655.0,125.0,1785.0
4,1948-01-01,New Brunswick,10.0,209.0,14.0,531.0
...,...,...,...,...,...,...
11981,2024-10-01,Quebec,12255.0,NaN,15814.0,NaN
11982,2024-10-01,Saskatchewan,12258.0,NaN,1184.0,NaN
11983,2024-10-01,Newfoundland and Labrador,12251.0,NaN,473.0,NaN
11984,2024-10-01,Atlantic provinces,NaN,NaN,NaN,NaN


In [21]:
# Define the list of GEO values to keep
geo_list = [
    "Canada", "Quebec", "Ontario", "British Columbia", "Alberta", 
    "Manitoba", "New Brunswick", "Newfoundland and Labrador", 
    "Nova Scotia", "Saskatchewan", "Prince Edward Island"
]

# Filter the DataFrame to keep only these GEO values
grouped_df = grouped_df[grouped_df['GEO'].isin(geo_list)].reset_index(drop=True)

In [22]:
grouped_df

,REF_DATE,GEO,Index,Housing completions,Housing starts,Housing under construction
0,1948-01-01,Alberta,25.0,855.0,250.0,1855.0
1,1948-01-01,British Columbia,28.0,2189.0,2121.0,6628.0
2,1948-01-01,Canada,1.0,11569.0,6678.0,37529.0
3,1948-01-01,Manitoba,19.0,655.0,125.0,1785.0
4,1948-01-01,New Brunswick,10.0,209.0,14.0,531.0
...,...,...,...,...,...,...
10137,2024-10-01,Ontario,12256.0,NaN,16287.0,NaN
10138,2024-10-01,Prince Edward Island,12252.0,NaN,357.0,NaN
10139,2024-10-01,Quebec,12255.0,NaN,15814.0,NaN
10140,2024-10-01,Saskatchewan,12258.0,NaN,1184.0,NaN


In [ ]:
#Filling up with 0
grouped_df.fillna(0, inplace=True)


In [30]:
# Function to distribute values for missing months
def distribute_values(group):
    group = group.set_index("REF_DATE").asfreq("MS")  # Set monthly frequency
    
    cols_to_fill = ["Housing completions", "Housing starts", "Housing under construction"]
    
    for col in cols_to_fill:
        non_null_dates = group[col].dropna().index  # Get dates where data exists
        
        for date in non_null_dates:
            if date.month in [1, 4, 7, 10]:  # Check if it's a quarter start (original months)
                value = group.loc[date, col]  # Get the original value
                
                # Calculate equal split for the next two months
                split_value = value / 3
                
                # Assign split values for that quarter
                group.at[date, col] = split_value
                if date + pd.DateOffset(months=1) in group.index:
                    group.at[date + pd.DateOffset(months=1), col] = split_value
                if date + pd.DateOffset(months=2) in group.index:
                    group.at[date + pd.DateOffset(months=2), col] = split_value
    
    return group.reset_index()

In [31]:
grouped_df_filled = grouped_df.groupby("GEO", group_keys=False).apply(distribute_values)

C:\Users\mihir\AppData\Local\Temp\ipykernel_20300\313050593.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df_filled = grouped_df.groupby("GEO", group_keys=False).apply(distribute_values)


In [32]:
grouped_df_filled

,REF_DATE,GEO,Index,Housing completions,Housing starts,Housing under construction
0,1948-01-01,Alberta,25.0,285.0,83.333333,618.333333
1,1948-02-01,Alberta,0.0,285.0,83.333333,618.333333
2,1948-03-01,Alberta,0.0,285.0,83.333333,618.333333
3,1948-04-01,Alberta,55.0,372.0,1068.333333,1354.666667
4,1948-05-01,Alberta,0.0,372.0,1068.333333,1354.666667
...,...,...,...,...,...,...
917,2024-06-01,Saskatchewan,0.0,0.0,310.666667,0.000000
918,2024-07-01,Saskatchewan,12247.0,0.0,499.666667,0.000000
919,2024-08-01,Saskatchewan,0.0,0.0,499.666667,0.000000
920,2024-09-01,Saskatchewan,0.0,0.0,499.666667,0.000000


In [33]:
# Filter data to start from 1976
grouped_df_filled = grouped_df_filled[grouped_df_filled['REF_DATE'] >= '1976-01-01'].reset_index(drop=True)

In [34]:
#Dropping the Index column
grouped_df_filled.drop(columns=['Index'], inplace=True)

In [36]:
grouped_df_filled

,REF_DATE,GEO,Housing completions,Housing starts,Housing under construction
0,1976-01-01,Alberta,1244.333333,2286.666667,6568.333333
1,1976-02-01,Alberta,1244.333333,2286.666667,6568.333333
2,1976-03-01,Alberta,1244.333333,2286.666667,6568.333333
3,1976-04-01,Alberta,2931.333333,3413.000000,7038.666667
4,1976-05-01,Alberta,2931.333333,3413.000000,7038.666667
...,...,...,...,...,...
6441,2024-06-01,Saskatchewan,0.000000,310.666667,0.000000
6442,2024-07-01,Saskatchewan,0.000000,499.666667,0.000000
6443,2024-08-01,Saskatchewan,0.000000,499.666667,0.000000
6444,2024-09-01,Saskatchewan,0.000000,499.666667,0.000000


In [35]:
grouped_df_filled.to_csv("HousingStarted_Manipulated.csv", index=False)